# Métricas do reprocessamento

Código feito por Igor Laltuf, ago/23

---

## Objetivo

Analisar o impacto do reprocessamento das viagens da linha 844 entre 01/06/2022 até 31/12/2022

## Etapas
Comparar valores pré e pós reprocessamento segundo as seguintes variáveis:

- A quantidade de viagens completas.

- A quilometragem total das viagens completas.

- O valor do subsídio a ser pago.

## Conclusão

Sobre o período entre 01/06/2022 e 31/12/2022:

- A quantidade de viagens completas aumentou de 1.920 para 3.258 viagens após o reprocessamento.

- A quilometragem total das viagens completas aumentou de 13.033 km para 22.122 km, um aumento de 69,73%.

- O valor do subsídio a ser pago subiu de R$ 5.237,70 para R$ 38.162,45




### Carregar bibliotecas

In [1]:
#!pip install matplotlib
import basedosdados as bd
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import folium
from datetime import timedelta, datetime
from shapely import wkt

# Paths
import os
import sys
from pathlib import Path 

current_path = Path().resolve()

paths = dict()
paths["raw"] = current_path / 'data' / 'raw'
paths["treated"] = current_path / 'data' / 'treated'

for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)

# Seta o projeto a ser cobrado
bd.config.billing_project_id = "rj-smtr-dev"

# Seta as configurações a serem usadas
bd.config.project_config_path = "C:/Users/igorl/.basedosdados/"
# bd.config.project_config_path = "D:\\basedosdados\\dev"

### 1. Comparar a variação na km após o reprocessamento

In [2]:
q = f"""
SELECT
  FORMAT_DATETIME('%Y-%m', DATA) AS mes_ano,
  SUM(ROUND(distancia_planejada,2)) AS distancia_total
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_completa`
WHERE
  DATA BETWEEN "2022-06-01"
  AND "2022-12-31"
  AND servico_informado = "844"
GROUP BY
  mes_ano
"""

viagens_realizadas_844 = bd.read_sql(q, from_file=True)

viagens_realizadas_844.to_csv('data/treated/viagens_realizadas_844_jun_22-dez_22.csv', index=False)

Downloading: 100%|██████████| 7/7 [00:00<00:00, 18.37rows/s]


In [3]:
q = f"""
SELECT
  FORMAT_DATETIME('%Y-%m', DATA) AS mes_ano,
  SUM(ROUND(distancia_planejada,2)) AS distancia_total
FROM
  `rj-smtr-dev.SMTR202211008775_reprocessamento.viagem_completa`
WHERE
  DATA BETWEEN "2022-06-01"
  AND "2022-12-31"
GROUP BY
  mes_ano
"""
viagens_reprocessadas_844 = bd.read_sql(q, from_file=True)

viagens_reprocessadas_844.to_csv('data/treated/viagens_reprocessadas_844_jun_22-dez_22.csv', index=False)

Downloading: 100%|██████████| 7/7 [00:00<00:00, 17.54rows/s]


In [4]:
viagens_reprocessadas_844 = pd.read_csv('data/treated/viagens_reprocessadas_844_jun_22-dez_22.csv')
viagens_realizadas_844 = pd.read_csv('data/treated/viagens_realizadas_844_jun_22-dez_22.csv')

km_mes_ano = pd.merge(viagens_realizadas_844, viagens_reprocessadas_844, how = 'left', on='mes_ano', suffixes=['','_reprocessada'])
km_mes_ano = km_mes_ano.sort_values('mes_ano')
km_mes_ano

,mes_ano,distancia_total,distancia_total_reprocessada
3,2022-06,2566.62,3035.13
4,2022-07,2454.36,3877.09
5,2022-08,2539.46,4209.80
0,2022-09,1425.90,2865.38
1,2022-10,1378.37,2946.86
6,2022-11,1249.36,2553.04
2,2022-12,1419.11,2634.52


In [5]:
distancia_total_pre_reprocessamento = km_mes_ano['distancia_total'].sum()
distancia_total_pos_reprocessamento = km_mes_ano['distancia_total_reprocessada'].sum()
print("Comparação da quilometragem das viagens completas")
print("Valor total da km pré-reprocessamento:", distancia_total_pre_reprocessamento)
print("Valor total da km pós-reprocessamento:", distancia_total_pos_reprocessamento)
print("Aumento de ",round((distancia_total_pos_reprocessamento / distancia_total_pre_reprocessamento - 1) * 100, 2),"%")

Comparação da quilometragem das viagens completas
Valor total da km pré-reprocessamento: 13033.180000000004
Valor total da km pós-reprocessamento: 22121.820000000003
Aumento de  69.73 %


In [34]:
# Criar o gráfico de linha
fig = go.Figure()

# Adicionar a linha com pontos para km_mes_ano['distancia_total']
fig.add_trace(go.Scatter(x=km_mes_ano['mes_ano'], y=km_mes_ano['distancia_total'],
                         mode='lines+markers',
                         name='Km pré-reprocessamento',
                         marker=dict(color='dodgerblue'),
                         opacity=0.75))

# Adicionar a linha com pontos para km_mes_ano['distancia_total_reprocessada']
fig.add_trace(go.Scatter(x=km_mes_ano['mes_ano'], y=km_mes_ano['distancia_total_reprocessada'],
                         mode='lines+markers',
                         name='Km reprocessada',
                         marker=dict(color='crimson'),
                         opacity=0.75))

# Configurar layout e outros parâmetros do gráfico
fig.update_layout(
    title=dict(text="Quilometragem apurada por mês - Serviço: 844", font=dict(color='black')),
    showlegend=False,
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Valores',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.025,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pré-reprocessamento',
            showarrow=False,
            font=dict(color='dodgerblue', size=14)
        ),
        dict(
            x=0.27,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='e',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.29,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pós-reprocessamento.',
            showarrow=False,
            font=dict(color='crimson', size=14)
        )
    ]
)

fig.show()

### 2. Comparar a variação no valor do subsídio após o reprocessamento

2.1 Valor recebido pelos consórcios antes do reprocessamento

In [7]:
q = f"""
SELECT
  CONCAT(CAST(EXTRACT(YEAR FROM DATA) AS STRING), '-', LPAD(CAST(EXTRACT(MONTH FROM DATA) AS STRING), 2, '0')) AS mes_ano,
  SUM(viagens) as viagens,
  
  SUM(valor_subsidio_pago) AS valor_subsidio
FROM
  `rj-smtr.dashboard_subsidio_sppo.sumario_servico_dia_historico`
WHERE
  servico = '844'
  AND DATA BETWEEN '2022-06-01' AND '2022-12-31'
GROUP BY
  mes_ano
"""
subsidio_pago_mes = bd.read_sql(q, from_file=True)


Downloading: 100%|██████████| 7/7 [00:00<00:00, 17.95rows/s]


In [8]:
subsidio_pago_mes = subsidio_pago_mes.sort_values('mes_ano')
subsidio_pago_mes = subsidio_pago_mes[['mes_ano', 'viagens', 'valor_subsidio']]
subsidio_pago_mes

,mes_ano,viagens,valor_subsidio
0,2022-06,378,4397.31
2,2022-07,362,0.00
6,2022-08,374,0.00
3,2022-09,210,130.71
5,2022-10,203,439.99
4,2022-11,184,124.66
1,2022-12,209,145.03


2.2 Valor que deve ser pago após o reprocessamento

Com base nas viagens identificadas na tabela de viagem completa que foi reprocessada, foram verificados aqueles dias em que POD >= 80%, para então calcular o valor do subsídio de acordo com os valores por km da tabela `subsidio_data_versao_efetiva` que está no dataset `rj-smtr-dev.SMTR202211008775_reprocessamento`.

Vale destacar que esta tabela exclui as glosas de 2023 do valor do subsídio por km. Ex: considera 2.81 por km após 16 de janeiro de 2023. 

In [9]:
q = f"""
WITH
  viagem_dia AS (
  SELECT
    DATA,
    COUNT(*) AS viagens,
    SUM(distancia_planejada) AS km_apurada
  FROM
    `rj-smtr-dev.SMTR202211008775_reprocessamento.viagem_completa`
  WHERE
    DATA BETWEEN "2022-06-01"
    AND "2022-12-31"
  GROUP BY
    DATA
  ),
  viagem_planejada AS ( -- pegar a km planejada para o dia
  SELECT DISTINCT 
    DATA,
    distancia_total_planejada
  FROM
    `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
  WHERE
    servico = '844'
    AND DATA BETWEEN "2022-06-01"
    AND "2022-12-31"
  ),
  tabela_pod AS (
  SELECT
    *,
    ROUND((km_apurada / distancia_total_planejada),2) AS perc_km_planejada
  FROM
    viagem_dia vd
  RIGHT JOIN
    viagem_planejada vp
  USING
    (DATA) 
  ),
  tabela_subsidio_km AS (
  SELECT
    DATA,
    valor_subsidio_por_km
  FROM
    `rj-smtr-dev.SMTR202211008775_reprocessamento.subsidio_data_versao_efetiva` )
SELECT
  DATA,
  COALESCE(viagens, 0) as viagens,
  COALESCE(km_apurada,0) as km_apurada,
  COALESCE(distancia_total_planejada,0) as distancia_total_planejada,
  COALESCE(perc_km_planejada,0) as perc_km_planejada,
  COALESCE(valor_subsidio_por_km,0) as valor_subsidio_por_km,
   CASE 
   WHEN perc_km_planejada >= 0.8 THEN valor_subsidio_por_km * km_apurada
   ELSE 0
   END as valor_subsidio
FROM
  tabela_pod
LEFT JOIN
  tabela_subsidio_km
USING
  (DATA)
"""

subsidio_a_pagar_dia = bd.read_sql(q, from_file=True)
subsidio_a_pagar_dia = subsidio_a_pagar_dia.sort_values('DATA')
subsidio_a_pagar_dia

Downloading: 100%|██████████| 214/214 [00:00<00:00, 518.16rows/s]


,DATA,viagens,km_apurada,distancia_total_planejada,perc_km_planejada,valor_subsidio_por_km,valor_subsidio
166,2022-06-01,29,196.91,129.020,1.53,2.13,419.4183
195,2022-06-02,23,156.17,129.020,1.21,2.13,332.6421
57,2022-06-03,22,149.38,129.020,1.16,2.13,318.1794
1,2022-06-04,0,0.00,80.638,0.00,2.13,0.0000
130,2022-06-05,0,0.00,64.510,0.00,2.13,0.0000
...,...,...,...,...,...,...,...
213,2022-12-27,20,135.80,135.806,1.00,1.78,241.7240
75,2022-12-28,21,142.59,135.806,1.05,1.78,253.8102
190,2022-12-29,21,142.59,135.806,1.05,1.78,253.8102
13,2022-12-30,20,135.80,135.806,1.00,1.78,241.7240


In [10]:
subsidio_a_pagar_dia['DATA'] = pd.to_datetime(subsidio_a_pagar_dia['DATA'])
subsidio_a_pagar_dia['mes_ano'] = subsidio_a_pagar_dia['DATA'].dt.strftime('%Y-%m')
subsidio_a_pagar_mes = subsidio_a_pagar_dia[['mes_ano', 'viagens', 'valor_subsidio']]
subsidio_a_pagar_mes = subsidio_a_pagar_mes.groupby('mes_ano').sum().reset_index()  
subsidio_a_pagar_mes

,mes_ano,viagens,valor_subsidio
0,2022-06,447,6262.3491
1,2022-07,571,6858.9864
2,2022-08,620,7369.8660
3,2022-09,422,5014.4150
4,2022-10,434,4619.9160
5,2022-11,376,3552.9354
6,2022-12,388,4483.9802


O aumento do valor do subsídio não é proporcional ao aumento da quantidade de viagens, pois com o reprocessamento, cada viagem circular virou duas viagens separadas, uma de ida e outra de volta.

Isto quer dizer que o esperado era que a quantidade de viagens pelo menos dobrasse após o reprocessamento, mesmo que não fosse identificada nenhuma viagem nova.

In [11]:
# juntar as duas tabelas
tabela_subsidio_mes = pd.merge(subsidio_pago_mes, subsidio_a_pagar_mes, how='left', on='mes_ano', suffixes=['_pre','_pos'])
tabela_subsidio_mes

,mes_ano,viagens_pre,valor_subsidio_pre,viagens_pos,valor_subsidio_pos
0,2022-06,378,4397.31,447,6262.3491
1,2022-07,362,0.00,571,6858.9864
2,2022-08,374,0.00,620,7369.8660
3,2022-09,210,130.71,422,5014.4150
4,2022-10,203,439.99,434,4619.9160
5,2022-11,184,124.66,376,3552.9354
6,2022-12,209,145.03,388,4483.9802


In [12]:
tabela_subsidio_mes
total_subsidio_comparacao = tabela_subsidio_mes[['viagens_pre', 'viagens_pos','valor_subsidio_pre','valor_subsidio_pos']].sum(axis=0)
total_subsidio_comparacao

viagens_pre            1920.0000
viagens_pos            3258.0000
valor_subsidio_pre     5237.7000
valor_subsidio_pos    38162.4481
dtype: float64

In [13]:
# Criar o gráfico de linha com plotly
fig = go.Figure()

# Adicionar a linha com pontos para tabela_subsidio_mes['viagens_pre']
fig.add_trace(go.Scatter(x=tabela_subsidio_mes['mes_ano'], y=tabela_subsidio_mes['viagens_pre'],
                         mode='lines+markers',
                         name='Qtd de viagens pré-reprocessamento',
                         marker=dict(color='dodgerblue'),
                         opacity=0.75))

# Adicionar a linha com pontos para tabela_subsidio_mes['viagens_pos']
fig.add_trace(go.Scatter(x=tabela_subsidio_mes['mes_ano'], y=tabela_subsidio_mes['viagens_pos'],
                         mode='lines+markers',
                         name='Qtd de viagens reprocessadas',
                         marker=dict(color='crimson'),
                         opacity=0.75))

# Configurar layout e outros parâmetros do gráfico seguindo o design anterior
fig.update_layout(
    title=dict(text="Quantidade de viagens por mês - Serviço: 844", font=dict(color='black')),
    showlegend=False,
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Valores',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.025,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pré-reprocessamento',
            showarrow=False,
            font=dict(color='dodgerblue', size=14)
        ),
        dict(
            x=0.27,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='e',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.29,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pós-reprocessamento.',
            showarrow=False,
            font=dict(color='crimson', size=14)
        )
    ]
)

fig.show()

In [24]:
# Criar o gráfico de linha com plotly
fig = go.Figure()

# Adicionar a linha com pontos para tabela_subsidio_mes['valor_subsidio_pre']
fig.add_trace(go.Scatter(x=tabela_subsidio_mes['mes_ano'], y=tabela_subsidio_mes['valor_subsidio_pre'],
                         mode='lines+markers',
                         name='Valor do subsídio pré-reprocessamento',
                         marker=dict(color='dodgerblue'),
                         opacity=0.75))

# Adicionar a linha com pontos para tabela_subsidio_mes['valor_subsidio_pos']
fig.add_trace(go.Scatter(x=tabela_subsidio_mes['mes_ano'], y=tabela_subsidio_mes['valor_subsidio_pos'],
                         mode='lines+markers',
                         name='Valor do subsídio reprocessado',
                         marker=dict(color='crimson'),
                         opacity=0.75))

# Configurar layout e outros parâmetros do gráfico seguindo o design anterior
fig.update_layout(
    title=dict(text="Valor do subsídio por mês - Serviço: 844", font=dict(color='black')),
    showlegend=False,
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Valores',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.025,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pré-reprocessamento',
            showarrow=False,
            font=dict(color='dodgerblue', size=14)
        ),
        dict(
            x=0.27,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='e',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.29,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pós-reprocessamento.',
            showarrow=False,
            font=dict(color='crimson', size=14)
        )
    ]
)

fig.show()

### 3. Análise do POD


3.1 POD pré-reprocessamento

In [25]:
# POD pré-reprocessamento
q = f"""
SELECT
  data, viagens, perc_km_planejada
FROM
  `rj-smtr.dashboard_subsidio_sppo.sumario_servico_dia_historico`
WHERE
  servico = '844'
  AND DATA BETWEEN '2022-06-01' AND '2022-12-31'
"""
pod_pre_reprocessamento = bd.read_sql(q, from_file=True)

Downloading: 100%|██████████| 214/214 [00:00<00:00, 621.92rows/s]


In [26]:
# Assegurando que a coluna 'data' seja do tipo datetime
pod_pre_reprocessamento = pod_pre_reprocessamento.sort_values('data')
pod_pre_reprocessamento['data'] = pd.to_datetime(pod_pre_reprocessamento['data'])
# Criando a nova coluna 'mes_ano'
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['data'].dt.to_period('M')
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['mes_ano'].astype(str)
pod_pre_reprocessamento

,data,viagens,perc_km_planejada,mes_ano
16,2022-06-01,25,131.59,2022-06
15,2022-06-02,25,131.59,2022-06
5,2022-06-03,17,89.48,2022-06
111,2022-06-04,0,0.00,2022-06
121,2022-06-05,0,0.00,2022-06
...,...,...,...,...
134,2022-12-27,11,55.00,2022-12
179,2022-12-28,13,65.00,2022-12
208,2022-12-29,9,45.00,2022-12
17,2022-12-30,11,55.00,2022-12


In [28]:
y = "perc_km_planejada"
title = f"Distribuição do percentual de operação diária (POD) por mês - Serviço: 844"

fig = px.box(
    pod_pre_reprocessamento, 
    x="mes_ano", 
    y=y, 
    title=title,
    template="plotly_white",
    labels={
        "mes_ano": "",
        y: ""
    },
    points="all"
)

fig.update_layout(
    yaxis_range=[-10, 200],  
    yaxis_ticksuffix="%",
    width=800,
    height=600,
    showlegend=False,
)

fig.add_hline(y=80, annotation_text="min = 80%")  # Linha horizontal para indicar o mínimo de 80%

fig.show()


3.2 POD pós-reprocessamento

In [29]:
pod_pos_reprocessamento = subsidio_a_pagar_dia[["mes_ano","viagens","perc_km_planejada"]]
pod_pos_reprocessamento['perc_km_planejada'] = pod_pos_reprocessamento['perc_km_planejada'] * 100
pod_pos_reprocessamento

,mes_ano,viagens,perc_km_planejada
166,2022-06,29,153.0
195,2022-06,23,121.0
57,2022-06,22,116.0
1,2022-06,0,0.0
130,2022-06,0,0.0
...,...,...,...
213,2022-12,20,100.0
75,2022-12,21,105.0
190,2022-12,21,105.0
13,2022-12,20,100.0


In [30]:

y = "perc_km_planejada"
title = "Distribuição do percentual de operação diária (POD) por mês - Serviço: 844 <br>Versão reprocessada"

fig = px.box(
    pod_pos_reprocessamento, 
    x="mes_ano", 
    y=y, 
    title=title,
    template="plotly_white",
    labels={
        "mes_ano": "",
        y: ""
    },
    points="all"
)

fig.update_layout(
    yaxis_range=[-50, 230],  
    yaxis_ticksuffix="%",
    width=800,
    height=600,
    showlegend=False,
)

fig.add_hline(y=80, annotation_text="min = 80%")  # Linha horizontal para indicar o mínimo de 80%

fig.show()


Consolidar POD pré e pós reprocessamento em apenas um gráfico.

In [31]:
# Calcular a média e o desvio padrão de perc_km_planejada por mês para ambos os DataFrames
media_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].mean()
std_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].std()

media_pos = pod_pos_reprocessamento.groupby('mes_ano')['perc_km_planejada'].mean()
std_pos = pod_pos_reprocessamento.groupby('mes_ano')['perc_km_planejada'].std()

Com o reprocessamento, houve uma melhoria do valor do Percentual de Operação Diária.

In [32]:
maiores_perc_pos = subsidio_a_pagar_dia.sort_values(by='perc_km_planejada', ascending=False)
maiores_perc_pos

,DATA,viagens,km_apurada,distancia_total_planejada,perc_km_planejada,valor_subsidio_por_km,valor_subsidio,mes_ano
197,2022-10-02,24,162.96,74.693,2.18,1.62,263.9952,2022-10
117,2022-10-30,21,142.59,74.693,1.91,1.62,230.9958,2022-10
184,2022-09-02,19,129.01,80.635,1.60,1.75,225.7675,2022-09
157,2022-12-09,21,142.59,93.366,1.53,1.78,253.8102,2022-12
166,2022-06-01,29,196.91,129.020,1.53,2.13,419.4183,2022-06
...,...,...,...,...,...,...,...,...
159,2022-10-15,0,0.00,93.367,0.00,1.62,0.0000,2022-10
123,2022-08-07,0,0.00,101.858,0.00,1.80,0.0000,2022-08
172,2022-10-12,0,0.00,74.693,0.00,1.62,0.0000,2022-10
16,2022-10-09,0,0.00,74.693,0.00,1.62,0.0000,2022-10


In [35]:
# Criar o gráfico de linha
fig = go.Figure()

# Adicionar a linha com pontos e desvio padrão para pod_pre_reprocessamento
fig.add_trace(go.Scatter(x=media_pre.index, y=media_pre.values,
                    mode='lines+markers',
                    name='Antes do Reprocessamento',
                    marker=dict(color='dodgerblue'),
                    opacity=0.75,
                    error_y=dict(type='data', array=std_pre, visible=True, color='dodgerblue')  # Corrigindo a cor aqui
))

# Adicionar a linha com pontos e desvio padrão para pod_pos_reprocessamento
fig.add_trace(go.Scatter(x=media_pos.index, y=media_pos.values,
                    mode='lines+markers',
                    name='Após Reprocessamento',
                    marker=dict(color='crimson'),
                    opacity=0.75,
                    error_y=dict(type='data', array=std_pos, visible=True, color='crimson')  # Corrigindo a cor aqui
))

# Configurar layout e outros parâmetros do gráfico
fig.update_layout(
    title=dict(text="Percentual de Operação Diária (POD) por mês - Serviço: 844", font=dict(color='black')),
    showlegend=False,
    yaxis_range=[0, 230],
    yaxis_ticksuffix="%",
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Valor médio e desvio-padrão mensais do POD',
            showarrow=False,
            font=dict(color='black', size=14)  # Aplicando a cor do título
        ),
        dict(
            x=0.445,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='antes do reprocessamento',
            showarrow=False,
            font=dict(color='dodgerblue', size=14)
        ),
        dict(
            x=0.745,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='e',
            showarrow=False,
            font=dict(color='black', size=14)  # Aplicando a cor do título
        ),
        dict(
            x=0.765,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='após o reprocessamento.',
            showarrow=False,
            font=dict(color='crimson', size=14)
        )
    ]
)

# Adicionar linha horizontal para indicar o mínimo de 80%
fig.add_hline(y=80, annotation_text="min = 80%")

fig.show()

### 4. Exportar planilha com dados

Exportar planilha resumo com valores a pagar antes e depois, assim como a km identificada na planilha de viagens completas.

In [23]:
km_mes_ano_soma = km_mes_ano[['distancia_total', 'distancia_total_reprocessada']].sum()
km_mes_ano_soma = km_mes_ano_soma.to_frame().T

km_mes_ano_soma = km_mes_ano_soma.rename(columns={
    'distancia_total': 'distancia_total_pre',
    'distancia_total_reprocessada': 'distancia_total_pos'
})
km_mes_ano_soma


tabela_subsidio_mes_soma = tabela_subsidio_mes[['viagens_pre', 'viagens_pos','valor_subsidio_pre', 'valor_subsidio_pos']].sum()
tabela_subsidio_mes_soma = tabela_subsidio_mes_soma.to_frame().T
tabela_subsidio_mes_soma

km_mes_ano_soma['key'] = 1
tabela_subsidio_mes_soma['key'] = 1

tabela_comparacao_exportar = km_mes_ano_soma.merge(tabela_subsidio_mes_soma, on='key', how='left').drop('key', axis=1)

tabela_comparacao_exportar
tabela_comparacao_exportar.to_excel("data/output/comparacao_pre_pos_reprocessamento_884.xlsx", index=False, engine='openpyxl')